# SparkContext and RDD basics

### Tirthajyoti Sarkar, Sunnyvale, CA, Oct 2018

### Import libraries

In [2]:
from pyspark import SparkContext
import numpy as np

### Initialize a `SparkContext` (the main abstraction to the cluster)
**Note the '4' in the argument. It denotes 4 cores to be used for this SparkContext object.**

In [3]:
sc=SparkContext(master="local[4]")

In [4]:
print(sc)

<SparkContext master=local[4] appName=pyspark-shell>


### Generate a list of random integeres

In [5]:
lst=np.random.randint(0,10,20)

In [6]:
print(lst)

[4 6 8 2 7 8 4 9 9 1 3 9 1 8 1 7 5 2 2 7]


### Parallelize the list - this is the main operation toward distributed computing

In [8]:
A=sc.parallelize(lst)

### What did we just do? We created a RDD? What is a RDD?
![](https://i.stack.imgur.com/cwrMN.png)
Spark revolves around the concept of a resilient distributed dataset (RDD), which is a **fault-tolerant collection of elements that can be operated on in parallel**. SparkContext manages the distributed data over the worker nodes through the cluster manager. 

There are two ways to create RDDs: 
* parallelizing an existing collection in your driver program, or 
* referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

We created a RDD using the former approach

### `A` is a pyspark RDD object, we cannot access the elements directly

In [9]:
type(A)

pyspark.rdd.RDD

In [10]:
A

ParallelCollectionRDD[1] at parallelize at PythonRDD.scala:184

### Opposite to parallelization - `collect` brings all the distributed elements and returns them to the head node. <br><br>Note - this is a slow process, do not use it often. 

In [11]:
A.collect()

[4, 6, 8, 2, 7, 8, 4, 9, 9, 1, 3, 9, 1, 8, 1, 7, 5, 2, 2, 7]

### How were the partitions created? Use `glom` method

In [12]:
A.glom().collect()

[[4, 6, 8, 2, 7], [8, 4, 9, 9, 1], [3, 9, 1, 8, 1], [7, 5, 2, 2, 7]]

### Now stop the SC and reinitialize it with 2 cores and see what happens when you repeat the process!

In [13]:
sc.stop()

In [14]:
sc=SparkContext(master="local[2]")

In [15]:
A = sc.parallelize(lst)

In [16]:
A.glom().collect()

[[4, 6, 8, 2, 7, 8, 4, 9, 9, 1], [3, 9, 1, 8, 1, 7, 5, 2, 2, 7]]

**The RDD is now distributed over two chunks, not four!** 

So, let's redo the process with 4 cores again.

In [17]:
sc.stop()

In [18]:
sc = SparkContext(master="local[4]")

In [19]:
A = sc.parallelize(lst)

### Count the elements

In [20]:
A.count()

20

### The first element (`first`) and the first few elements (`take`)

In [41]:
A.first()

3

In [42]:
A.take(4)

[3, 3, 7, 8]

### Get another RDD with only the distinct elements

In [43]:
A_distinct=A.distinct()

In [44]:
A_distinct.collect()

[8, 4, 0, 1, 5, 9, 6, 2, 3, 7]

### To sum all the elements use `reduce` method

In [45]:
A.reduce(lambda x,y:x+y)

102

### Or direct `sum` method

In [46]:
A.sum()

102

### Or using the `fold` method, which aggregates the elements of each partition, and then the results for all the partitions

In [47]:
A.fold(0,lambda x,y:x+y)

102

### Finding maximum element by `reduce`

In [48]:
A.reduce(lambda x,y: x if x > y else y)

9

### Finding longest word using `reduce`

In [42]:
words = 'These are some of the best Macintosh computers ever'.split(' ')
wordRDD = sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)>len(v) else v)

'computers'

### Use `filter` to return a new RDD with elements satisfying a given predicate (lambda expression)

In [49]:
# Return RDD with elements divisible by 3
A.filter(lambda x:x%3==0).collect()

[3, 3, 6, 3, 9, 0, 9, 6]

### Lambda functions are short and sweet but we can write regular Python functions to use with `reduce`

In [49]:
def largerThan(x,y):
    """
    Returns the last word among the longest words in a list
    """
    if len(x)> len(y):
        return x
    elif len(y) > len(x):
        return y
    else:
        if x < y: return x
        else: return y

In [50]:
wordRDD.reduce(largerThan)

'Macintosh'

### Basic statistics

In [51]:
print("Maximum: ",A.max())
print("Minimum: ",A.min())
print("Mean (average): ",A.mean())
print("Standard deviation: ",A.stdev())

Maximum:  9
Minimum:  1
Mean (average):  5.15
Standard deviation:  2.903015673398957


In [52]:
A.stats()

(count: 20, mean: 5.15, stdev: 2.903015673398957, max: 9.0, min: 1.0)

### `map` operation with _lambda_ function

In [53]:
B=A.map(lambda x:x*x)

In [54]:
B.collect()

[16, 36, 64, 4, 49, 64, 16, 81, 81, 1, 9, 81, 1, 64, 1, 49, 25, 4, 4, 49]

### `map` operation with regular Python function

In [55]:
def square_if_odd(x):
    if x%2==1:
        return x*x
    else:
        return x

In [56]:
A.map(square_if_odd).collect()

[4, 6, 8, 2, 49, 8, 4, 81, 81, 1, 9, 81, 1, 8, 1, 49, 25, 2, 2, 49]

### `flatmap` method returns a new RDD by first applying a function to all elements of this RDD, and then flattening the results

In [57]:
A.flatMap(lambda x:(x,x*x)).collect()

[4,
 16,
 6,
 36,
 8,
 64,
 2,
 4,
 7,
 49,
 8,
 64,
 4,
 16,
 9,
 81,
 9,
 81,
 1,
 1,
 3,
 9,
 9,
 81,
 1,
 1,
 8,
 64,
 1,
 1,
 7,
 49,
 5,
 25,
 2,
 4,
 2,
 4,
 7,
 49]

### `groupby` returns a RDD of grouped elements (iterable) as per a given group operation (function)

In [58]:
result=A.groupBy(lambda x:x%2).collect()
sorted([(x, sorted(y)) for (x, y) in result])

[(0, [2, 2, 2, 4, 4, 6, 8, 8, 8]), (1, [1, 1, 1, 3, 5, 7, 7, 7, 9, 9, 9])]

### `histogram` method takes a list of bins/buckets and returns a tuple with result of the histogram (binning) 

In [59]:
B.histogram([x for x in range(0,100,10)])

([0, 10, 20, 30, 40, 50, 60, 70, 80, 90], [7, 2, 1, 1, 3, 0, 3, 0, 3])

### Create smaller RDDs to demonstrate joint operations

In [60]:
lst1=np.random.randint(0,10,3)
C=sc.parallelize(lst1)
lst2=np.random.randint(10,20,3)
D=sc.parallelize(lst2)
print("C:",C.collect())
print("D:",D.collect())

C: [2, 5, 0]
D: [18, 18, 13]


### `C+D` gives the union (like set union), not the element wise sum

In [61]:
(C+D).collect()

[2, 5, 0, 18, 18, 13]

### `cartesian` gives the pairwise product (as tuples) 

In [62]:
C.cartesian(D).collect()

[(2, 18),
 (2, 18),
 (2, 13),
 (5, 18),
 (5, 18),
 (5, 13),
 (0, 18),
 (0, 18),
 (0, 13)]

### `intersection` and `subtract `methods return a RDD of the set intersection and subtraction (difference)

In [62]:
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()

[1, 2, 3]

In [63]:
rdd1.subtract(rdd2).collect()

[10, 4, 5]

### Stop the `SparkContext` at the end

In [64]:
sc.stop()